In [1]:
from seeq import spy
from seeq.spy.assets import Asset, ItemGroup
import pandas as pd

In [2]:
spy.login(username='Your Username Here', password='Your Password Here', url='Server URL')

In [15]:
class PID(Asset):
    @Asset.Component()
    def case(self, metadata:pd.DataFrame):
        return self.build_components(template=Case, metadata=metadata, column_name='Case')
    
class Case(Asset):
    # @Asset.Component()
    # def controller(self, metadata:pd.DataFrame):
    #     return self.build_components(template=Controller, metadata=metadata, column_name='Controller')  

    @Asset.Component()
    def model(self, metadata:pd.DataFrame):
        return self.build_components(template=Model, metadata=metadata, column_name='Model') 
    
class Model(Asset):
    @Asset.Attribute()
    def gain(self, metadata):
        return {
            'Name': 'gain',
            'Type': 'Scalar',
            'Formula': '$gain',
            'Formual Parameters': {
                '$gain': 100
            }
        }

In [16]:
url = 'http://prism-seeq-server:34216/123CF173-71EA-477E-86FC-EB5813C6F565/workbook/F6F0969A-07BC-414F-B183-D515CCE7652A/worksheet/B9854553-02B5-43F6-B08A-8B10C73C483A'
df = spy.search(
    url
)

# We can use Pandas' string extraction capabilities to create the Build Asset column
# df['Build Asset'] = pd.DataFrame()

# We will specify a simple path in a new tree where we want these to live
# df['Build Path'] = 'my HVAC Units'

df

,Name,ID,Type,Color,Line Style,Line Width,Lane,Samples Display,Axis Auto Scale,Axis Align,Axis Group,Axis Show,Time,Count,Pages,Result
0,F_cw,96215E67-A879-42A2-BA56-9D290DA1F602,Signal,#00A2DD,Solid,1.0,1.0,Line,True,Left,A,True,00:00:00.29,1,0,Success
1,T1,CD2897DD-A8E7-4558-8688-13BCC929AC89,Signal,#4055A3,Solid,1.0,2.0,Line,True,Left,B,True,00:00:00.29,1,0,Success
2,T2,34AB299D-C5C3-4551-8E31-702345885297,Signal,#068C45,Solid,1.0,3.0,Line,True,Left,C,True,00:00:00.28,1,0,Success
3,CA1,E47DCC3F-F571-4E8D-9809-EC8E3A3DE5F3,Signal,#9D248F,Solid,1.0,4.0,Line,True,Left,D,True,00:00:00.30,1,0,Success
4,CA2,BC3FE129-7290-427B-AFF7-F1843BD1F49B,Signal,#CE561B,Solid,1.0,5.0,Line,True,Left,E,True,00:00:00.28,1,0,Success
5,Train Condition,75773737-71A7-4D5B-AF10-A733D04D9089,Condition,#E1498E,nan,nan,6.0,nan,nan,nan,G,nan,00:00:00.58,1,0,Success
6,Validation Condition,041DAFCB-5307-4D76-8FD2-4DEFD3C85444,Condition,#996442,nan,nan,7.0,nan,nan,nan,F,nan,00:00:00.57,1,0,Success


,Name,Type,Scoped To,Archived,Cache Enabled,Cache ID,Cached By Service,Data ID,Datasource Class,Datasource ID,...,Source Value Unit Of Measure,Storage Location,Unsearchable,Value Unit Of Measure,Formula,Key Unit Of Measure,Maximum Duration,Metadata Properties,UIConfig,Formula Parameters
0,F_cw,StoredSignal,f6f0969a-07bc-414f-b183-d515cce7652a,False,False,a0cd6a8e-a8d3-4073-9f8e-212bc74929f7,true,3902ee3e-0f7c-42bc-b81e-6d8174640501-F_cw-SIGNAL,Imported CSV Files,Imported CSV Files,...,,FILE_SIGNAL_STORAGE,False,,NaN,NaN,NaN,NaN,NaN,NaN
1,T1,StoredSignal,f6f0969a-07bc-414f-b183-d515cce7652a,False,False,ad940a89-77e2-4b1a-b7e0-105060548365,true,3902ee3e-0f7c-42bc-b81e-6d8174640501-T1-SIGNAL,Imported CSV Files,Imported CSV Files,...,,FILE_SIGNAL_STORAGE,False,,NaN,NaN,NaN,NaN,NaN,NaN
2,T2,StoredSignal,f6f0969a-07bc-414f-b183-d515cce7652a,False,False,2270e9dc-aba4-43cf-8db9-6bf7ce2f9fb0,true,3902ee3e-0f7c-42bc-b81e-6d8174640501-T2-SIGNAL,Imported CSV Files,Imported CSV Files,...,,FILE_SIGNAL_STORAGE,False,,NaN,NaN,NaN,NaN,NaN,NaN
3,CA1,StoredSignal,f6f0969a-07bc-414f-b183-d515cce7652a,False,False,71273c5e-5cb8-4063-8539-2f97af3b6a2f,true,3902ee3e-0f7c-42bc-b81e-6d8174640501-CA1-SIGNAL,Imported CSV Files,Imported CSV Files,...,,FILE_SIGNAL_STORAGE,False,,NaN,NaN,NaN,NaN,NaN,NaN
4,CA2,StoredSignal,f6f0969a-07bc-414f-b183-d515cce7652a,False,False,f397676c-88eb-4284-b527-223460610e8b,true,3902ee3e-0f7c-42bc-b81e-6d8174640501-CA2-SIGNAL,Imported CSV Files,Imported CSV Files,...,,FILE_SIGNAL_STORAGE,False,,NaN,NaN,NaN,NaN,NaN,NaN
5,Train Condition,CalculatedCondition,f6f0969a-07bc-414f-b183-d515cce7652a,False,True,74dc52ca-9ef3-47e0-b13f-245c2d73b37d,NaN,75773737-71A7-4D5B-AF10-A733D04D9089,Seeq Calculations,Seeq Calculations,...,NaN,NaN,False,NaN,"condition(4886437.102294922ms,\n capsule(1609...",ns,4886437.102294922,,"{""type"":""custom-condition""}",[]
6,Validation Condition,CalculatedCondition,f6f0969a-07bc-414f-b183-d515cce7652a,False,True,e140f9c8-0898-4f19-a363-d946777551fb,NaN,041DAFCB-5307-4D76-8FD2-4DEFD3C85444,Seeq Calculations,Seeq Calculations,...,NaN,NaN,False,NaN,"condition(9480206.318115234ms,\n capsule(1609...",ns,9480206.318115234,,"{""type"":""custom-condition""}",[]


In [17]:
model_name = 'T1'
case_number = '#1'

df.at[df['Name']=='T1', 'Build Asset'] = 'PID_' + model_name
# We will specexy a simple path in a new tree where we want these to live
df.at[df['Name']=='T1', 'Build Path'] = 'PID Tuner'
df.at[df['Name']=='T1', ['Case', 'Model']] = ['Case#1', 'Model_'+model_name+'_'+case_number]

In [18]:
build_df = spy.assets.build(PID, df, errors='raise')

build_df

,Build Path,Build Asset,Build Template,Build Result
0,PID Tuner,PID_T1,PID,Success


,Name,Type,Formula,Formual Parameters,Asset,Asset Object,Path,Template,Build Result
0,gain,Scalar,$gain,{'$gain': 100},Model_T1_#1,PID Tuner >> PID_T1 >> Case#1 >> Model_T1_#1,PID Tuner >> PID_T1 >> Case#1,Model,Success
1,Model_T1_#1,Asset,NaN,NaN,Model_T1_#1,PID Tuner >> PID_T1 >> Case#1 >> Model_T1_#1,PID Tuner >> PID_T1 >> Case#1,Model,Success
2,Case#1,Asset,NaN,NaN,Case#1,PID Tuner >> PID_T1 >> Case#1,PID Tuner >> PID_T1,Case,Success
3,PID_T1,Asset,NaN,NaN,PID_T1,PID Tuner >> PID_T1,PID Tuner,PID,Success


In [ ]:
spy.push(metadata=build_df, workbook='F6F0969A-07BC-414F-B183-D515CCE7652A')


,ID,Name,Workbook Type,Count,Time,Result
0,F6F0969A-07BC-414F-B183-D515CCE7652A,Two Series Reactors,Analysis,1,00:00:02.68,Pulling workbook
